In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold

# Analysis Connectome

**Research question**: are there differences in brain connection patterns between men and women?

**Method**: Linear Discriminant Analysis: find a linear combination of features that characterizes or separates two or more class objects.
- We already know the groups, thus we can use this method as a classification function.

**LDA assumptions**:
- Independent variables are normally distributed
- Variances among gorup variables are same across levels of predictors.
- Independence: participants are assumed to be randomly sampled.

**Dataset**:
- Rows contain boolean indicating whether a connection between two brain areas is present or not. Only connections with variance among subjects are included.
- Last column indicates the sex of the participant.

In [5]:
df = pd.read_csv('../DATASET/connectome_vectorized.csv')
df.head()

Unnamed: 0  0  1  2  3  4  5  6  7  8  ...  93139  93140  93141  93142  \
0           0  1  0  0  0  0  0  0  0  0  ...      0      0      1      0   
1           1  0  0  0  0  1  0  1  0  0  ...      0      0      0      1   
2           2  0  0  1  0  0  0  0  0  0  ...      0      0      0      0   
3           3  0  0  0  0  1  0  1  0  0  ...      0      0      0      0   
4           4  0  0  0  0  0  0  0  0  0  ...      0      0      0      0   

   93143  93144  93145  93146  93147  sex  
0      0      1      0      0      0    F  
1      0      1      1      1      1    M  
2      0      1      0      0      1    M  
3      0      1      1      0      1    M  
4      0      1      1      0      1    F  

[5 rows x 93150 columns]

We can see that it contains a column 'Unnamed'. We remove that column.

In [9]:
df = df.drop(columns='Unnamed: 0')

KeyError: "['Unnamed: 0'] not found in axis"

In [10]:
df.head()

0  1  2  3  4  5  6  7  8  9  ...  93139  93140  93141  93142  93143  \
0  1  0  0  0  0  0  0  0  0  0  ...      0      0      1      0      0   
1  0  0  0  0  1  0  1  0  0  0  ...      0      0      0      1      0   
2  0  0  1  0  0  0  0  0  0  0  ...      0      0      0      0      0   
3  0  0  0  0  1  0  1  0  0  0  ...      0      0      0      0      0   
4  0  0  0  0  0  0  0  0  0  0  ...      0      0      0      0      0   

   93144  93145  93146  93147  sex  
0      1      0      0      0    F  
1      1      1      1      1    M  
2      1      0      0      1    M  
3      1      1      0      1    M  
4      1      1      0      1    F  

[5 rows x 93149 columns]

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Columns: 93149 entries, 0 to sex
dtypes: int64(93148), object(1)
memory usage: 68.2+ MB


Split the data in the booleans and the sex

In [13]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

print(X)
print(y)

[[1 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]
 [0 0 1 ... 0 0 1]
 ...
 [0 0 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [0 0 0 ... 1 0 0]]
['F' 'M' 'M' 'M' 'F' 'M' 'F' 'F' 'F' 'F' 'M' 'M' 'F' 'F' 'F' 'M' 'M' 'F'
 'M' 'F' 'M' 'M' 'M' 'M' 'M' 'F' 'F' 'F' 'F' 'F' 'M' 'F' 'F' 'M' 'M' 'F'
 'F' 'F' 'F' 'M' 'F' 'F' 'M' 'M' 'F' 'F' 'F' 'F' 'F' 'F' 'M' 'M' 'M' 'F'
 'M' 'F' 'F' 'M' 'M' 'M' 'M' 'F' 'F' 'M' 'F' 'M' 'F' 'M' 'F' 'F' 'F' 'M'
 'M' 'M' 'F' 'M' 'M' 'F' 'M' 'F' 'F' 'F' 'F' 'F' 'M' 'M' 'M' 'F' 'F' 'M'
 'M' 'M' 'F' 'F' 'M' 'F']


Take a look at the variance, to make sure the difference is not too big.

In [30]:
X.var(axis=0).max(), X.var(axis=0).min()

(0.25, 0.010308159722222219)

The variance is fine, it does not differ by much. Let's create the model.

In [8]:
model = LinearDiscriminantAnalysis()

Perform KFOLD, as it introduces less bias and less optimistic estimate.

In [9]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)

In [11]:
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

In [12]:
print('Mean accuracy: {:.3f} {:.3f}'.format(np.mean(scores), np.std(scores)))

Mean accuracy: 0.573 0.084


The possible accuracy is not optimistic. Lets try what LinearDiscriminantAnalysis gives us.

In [36]:
model.fit(X, y)

LinearDiscriminantAnalysis()

In [37]:
pred = model.predict(X)

In [38]:
accuracy = (y == pred).sum() / y.size
print('Accuracy: {:.3f}'.format(accuracy))

Accuracy: 0.677


**Interpretation and reflection**

The accuracy is above 0.5. This means there is a linear combination of variables that can differentiate M from F. However, this is much more optimistic then the chorss validation score. This may be due to overfitting or another 
factor that causes bias.

Both LDA and cross validation give an accuracy above 0.5. This means it is possible to discriminate men from women. We can conclude that there are differences in brain activity between men and women.

Another option was using a dimensionality reduction technique for the raw brain scans and use them as input for LDA.